In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('datasets/sentiment_chatgpt.csv', usecols=['user_name', 'user_description','tweets','label', 'date'])

In [6]:
df.isnull().values.any()
df.isnull().sum()

user_name           0
user_description    0
tweets              0
label               0
date                0
dtype: int64

In [7]:
df = df.sort_values(by='date')

In [8]:
df.date = pd.to_datetime(df.date)

In [9]:
first = df[df.date.dt.month == 12] # first monthes 

In [10]:
last = df[(df.date.dt.month == 4) ] # last month (april)

In [11]:
positiv = first[first.label == 'positiv']
negativ = first[first.label == 'negativ']

## Topic extraction

In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.9 MB/s eta 0:00:00


In [16]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

    
MODEL = f"cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = model.config.id2label

In [17]:
# topic modelling 

def get_topic(text): 
    tokenized_inputs = []
    for sentence in text:
        tokens = tokenizer(sentence, return_tensors='pt')
        tokenized_inputs.append(tokens)
        
    predictions = []
    for tokens in tokenized_inputs:
        output = model(**tokens)
        scores = output.logits[0].detach().numpy()
        scores = expit(scores)
        sentence_predictions = (scores >= 0.5) * 1
        predictions.append(sentence_predictions)
        
    classes = []
    for sentence_predictions in predictions:
        for i in range(len(sentence_predictions)):
            if sentence_predictions[i]:
                classes.append(class_mapping[i])
                
    return classes

### First month

In [ ]:
## topic modelling of all tweets
text = list(first.tweets)

In [ ]:
classes = get_topic(text)

In [ ]:
from collections import Counter

# Count the occurrences of each string
string_counts = Counter(classes)

for string, count in string_counts.items():
    print(f"Class: {string}, Count: {count}")

Class: learning_&_educational, Count: 6152
Class: news_&_social_concern, Count: 6394
Class: diaries_&_daily_life, Count: 6162
Class: other_hobbies, Count: 919
Class: science_&_technology, Count: 14700
Class: business_&_entrepreneurs, Count: 6128
Class: youth_&_student_life, Count: 1897
Class: arts_&_culture, Count: 1886
Class: gaming, Count: 994
Class: film_tv_&_video, Count: 2627
Class: sports, Count: 877
Class: celebrity_&_pop_culture, Count: 1323
Class: music, Count: 1155
Class: family, Count: 113
Class: relationships, Count: 308
Class: travel_&_adventure, Count: 655
Class: food_&_dining, Count: 427
Class: fashion_&_style, Count: 92
Class: fitness_&_health, Count: 406


In [ ]:
positiv = first[first.label == 'positiv']
negativ = first[first.label == 'negativ']

In [ ]:
positiv

,user_name,user_description,tweets,label,date
357638,abhas tandon,software engineer javascript nodejs python htm...,opinion mine criticism chatgpt generated chatg...,positiv,2022-12-05 17:12:28+00:00
357633,vice,little bit marketing little bit saas,weve seen extreme capability openais recent pr...,positiv,2022-12-05 17:14:20+00:00
357628,christopher adolphe,veloppeur frontend,wanted chatgpt try guess ill wait,positiv,2022-12-05 17:15:31+00:00
357625,pankaj joshi,data scientist yayavar photographer chess mani...,it mark critical thinking deep analytical task...,positiv,2022-12-05 17:16:08+00:00
357621,rob elli,cofounder ourtwostories trustee mfccommunity f...,thread chatgpt getting insane chatgpt,positiv,2022-12-05 17:16:55+00:00
...,...,...,...,...,...
305315,lakejoco,inspired meme forager rambler,podcast return chatgpt artificialintelligence ...,positiv,2022-12-31 23:56:28+00:00
305314,vinny carpenter,geek mission life strive seek find yield opini...,new favorite thing chatgpt create playlist her...,positiv,2022-12-31 23:58:31+00:00
305313,kawtar choubari,content creator software engineer tweet progra...,sound like cant predict trend newyear chatgpt ...,positiv,2022-12-31 23:58:37+00:00
305312,yordan dimitrov,focusing seo techcharities more tweeting lates...,asked chatgpt write nye joke seos delivered se...,positiv,2022-12-31 23:59:30+00:00


In [ ]:
len(positiv)

17319

In [ ]:
len(negativ)

18252

In [ ]:
from collections import Counter

# topic modelling for positive tweets
print("positive tweets topic modelling : ")
pos = list(positiv.tweets)
classes = get_topic(pos)

string_counts = Counter(classes)

fmap_pos = {}
for string, count in string_counts.items():
    fmap_pos[string] = count
    print(f"Class: {string}, Count: {count}")

sorted_fpos = sorted(fmap_pos.items(), key=lambda x: x[1])

positive tweets topic modelling : 
Class: science_&_technology, Count: 5968
Class: business_&_entrepreneurs, Count: 2253
Class: diaries_&_daily_life, Count: 1609
Class: learning_&_educational, Count: 2900
Class: gaming, Count: 333
Class: film_tv_&_video, Count: 829
Class: family, Count: 38
Class: relationships, Count: 102
Class: news_&_social_concern, Count: 2131
Class: music, Count: 359
Class: youth_&_student_life, Count: 975
Class: food_&_dining, Count: 135
Class: arts_&_culture, Count: 626
Class: sports, Count: 257
Class: celebrity_&_pop_culture, Count: 347
Class: travel_&_adventure, Count: 120
Class: fitness_&_health, Count: 127
Class: other_hobbies, Count: 221
Class: fashion_&_style, Count: 27


In [ ]:
sorted_fpos

[('fashion_&_style', 27),
 ('family', 38),
 ('relationships', 102),
 ('travel_&_adventure', 120),
 ('fitness_&_health', 127),
 ('food_&_dining', 135),
 ('other_hobbies', 221),
 ('sports', 257),
 ('gaming', 333),
 ('celebrity_&_pop_culture', 347),
 ('music', 359),
 ('arts_&_culture', 626),
 ('film_tv_&_video', 829),
 ('youth_&_student_life', 975),
 ('diaries_&_daily_life', 1609),
 ('news_&_social_concern', 2131),
 ('business_&_entrepreneurs', 2253),
 ('learning_&_educational', 2900),
 ('science_&_technology', 5968)]

In [ ]:
# topic modelling for negative tweets
print("negative tweets topic modelling : ")
text = list(negativ.tweets)
classes = get_topic(text)

string_counts = Counter(classes)

for string, count in string_counts.items():
    print(f"Class: {string}, Count: {count}")

negative tweets topic modelling : 
Class: diaries_&_daily_life, Count: 2759
Class: other_hobbies, Count: 444
Class: news_&_social_concern, Count: 2164
Class: business_&_entrepreneurs, Count: 1878
Class: science_&_technology, Count: 3898
Class: arts_&_culture, Count: 637
Class: gaming, Count: 319
Class: learning_&_educational, Count: 1406
Class: sports, Count: 328
Class: music, Count: 406
Class: film_tv_&_video, Count: 935
Class: celebrity_&_pop_culture, Count: 549
Class: travel_&_adventure, Count: 77
Class: food_&_dining, Count: 160
Class: fashion_&_style, Count: 37
Class: fitness_&_health, Count: 112
Class: youth_&_student_life, Count: 372
Class: relationships, Count: 115
Class: family, Count: 38


In [ ]:
string_counts

Counter({'diaries_&_daily_life': 2759,
         'other_hobbies': 444,
         'news_&_social_concern': 2164,
         'business_&_entrepreneurs': 1878,
         'science_&_technology': 3898,
         'arts_&_culture': 637,
         'gaming': 319,
         'learning_&_educational': 1406,
         'sports': 328,
         'music': 406,
         'film_tv_&_video': 935,
         'celebrity_&_pop_culture': 549,
         'travel_&_adventure': 77,
         'food_&_dining': 160,
         'fashion_&_style': 37,
         'fitness_&_health': 112,
         'youth_&_student_life': 372,
         'relationships': 115,
         'family': 38})

In [ ]:
fmap_neg = {}
for string, count in string_counts.items():
  fmap_neg[string] = count

In [ ]:
sorted_fneg = sorted(fmap_neg.items(), key=lambda x: x[1])

In [ ]:
sorted_fneg

[('fashion_&_style', 37),
 ('family', 38),
 ('travel_&_adventure', 77),
 ('fitness_&_health', 112),
 ('relationships', 115),
 ('food_&_dining', 160),
 ('gaming', 319),
 ('sports', 328),
 ('youth_&_student_life', 372),
 ('music', 406),
 ('other_hobbies', 444),
 ('celebrity_&_pop_culture', 549),
 ('arts_&_culture', 637),
 ('film_tv_&_video', 935),
 ('learning_&_educational', 1406),
 ('business_&_entrepreneurs', 1878),
 ('news_&_social_concern', 2164),
 ('diaries_&_daily_life', 2759),
 ('science_&_technology', 3898)]

### Last two monthes

In [12]:
## topic modelling of all tweets
text = list(last.tweets)

In [ ]:
classes = get_topic(text)

In [ ]:
from collections import Counter

# Count the occurrences of each string
string_counts = Counter(classes)

for string, count in string_counts.items():
    print(f"Class: {string}, Count: {count}")

In [18]:
positiv = last[last.label == 'positiv']
negativ = last[last.label == 'negativ']

In [21]:
from collections import Counter

# topic modelling for positive tweets
print("positive tweets topic modelling : ")
text = list(positiv.tweets)
classes = get_topic(text)

string_counts = Counter(classes)

for string, count in string_counts.items():
    print(f"Class: {string}, Count: {count}")

positive tweets topic modelling : 
Class: news_&_social_concern, Count: 6035
Class: science_&_technology, Count: 13491
Class: music, Count: 394
Class: diaries_&_daily_life, Count: 1716
Class: business_&_entrepreneurs, Count: 10655
Class: relationships, Count: 137
Class: travel_&_adventure, Count: 244
Class: film_tv_&_video, Count: 1233
Class: learning_&_educational, Count: 4426
Class: gaming, Count: 342
Class: celebrity_&_pop_culture, Count: 371
Class: fashion_&_style, Count: 95
Class: food_&_dining, Count: 213
Class: fitness_&_health, Count: 470
Class: youth_&_student_life, Count: 1388
Class: other_hobbies, Count: 349
Class: arts_&_culture, Count: 1000
Class: sports, Count: 331
Class: family, Count: 49


In [24]:
lmap_pos = {}

for string, count in string_counts.items():
    lmap_pos[string] = count
    print(f"Class: {string}, Count: {count}")

sorted_lpos = sorted(lmap_pos.items(), key=lambda x: x[1])

Class: news_&_social_concern, Count: 6035
Class: science_&_technology, Count: 13491
Class: music, Count: 394
Class: diaries_&_daily_life, Count: 1716
Class: business_&_entrepreneurs, Count: 10655
Class: relationships, Count: 137
Class: travel_&_adventure, Count: 244
Class: film_tv_&_video, Count: 1233
Class: learning_&_educational, Count: 4426
Class: gaming, Count: 342
Class: celebrity_&_pop_culture, Count: 371
Class: fashion_&_style, Count: 95
Class: food_&_dining, Count: 213
Class: fitness_&_health, Count: 470
Class: youth_&_student_life, Count: 1388
Class: other_hobbies, Count: 349
Class: arts_&_culture, Count: 1000
Class: sports, Count: 331
Class: family, Count: 49


In [25]:
sorted_lpos

[('family', 49),
 ('fashion_&_style', 95),
 ('relationships', 137),
 ('food_&_dining', 213),
 ('travel_&_adventure', 244),
 ('sports', 331),
 ('gaming', 342),
 ('other_hobbies', 349),
 ('celebrity_&_pop_culture', 371),
 ('music', 394),
 ('fitness_&_health', 470),
 ('arts_&_culture', 1000),
 ('film_tv_&_video', 1233),
 ('youth_&_student_life', 1388),
 ('diaries_&_daily_life', 1716),
 ('learning_&_educational', 4426),
 ('news_&_social_concern', 6035),
 ('business_&_entrepreneurs', 10655),
 ('science_&_technology', 13491)]

In [26]:
# topic modelling for negative tweets
print("negative tweets topic modelling : ")
text = list(negativ.tweets)
classes = get_topic(text)

string_counts = Counter(classes)

lmap_neg = {}
for string, count in string_counts.items():
    lmap_neg[string] = count
    print(f"Class: {string}, Count: {count}")

sorted_lneg = sorted(lmap_neg.items(), key=lambda x: x[1])

negative tweets topic modelling : 
Class: diaries_&_daily_life, Count: 2269
Class: relationships, Count: 228
Class: science_&_technology, Count: 7499
Class: news_&_social_concern, Count: 3729
Class: other_hobbies, Count: 461
Class: learning_&_educational, Count: 3186
Class: youth_&_student_life, Count: 775
Class: film_tv_&_video, Count: 1341
Class: business_&_entrepreneurs, Count: 6646
Class: fitness_&_health, Count: 290
Class: sports, Count: 306
Class: celebrity_&_pop_culture, Count: 506
Class: travel_&_adventure, Count: 247
Class: gaming, Count: 345
Class: music, Count: 285
Class: arts_&_culture, Count: 857
Class: fashion_&_style, Count: 93
Class: food_&_dining, Count: 212
Class: family, Count: 50


In [27]:
sorted_lneg

[('family', 50),
 ('fashion_&_style', 93),
 ('food_&_dining', 212),
 ('relationships', 228),
 ('travel_&_adventure', 247),
 ('music', 285),
 ('fitness_&_health', 290),
 ('sports', 306),
 ('gaming', 345),
 ('other_hobbies', 461),
 ('celebrity_&_pop_culture', 506),
 ('youth_&_student_life', 775),
 ('arts_&_culture', 857),
 ('film_tv_&_video', 1341),
 ('diaries_&_daily_life', 2269),
 ('learning_&_educational', 3186),
 ('news_&_social_concern', 3729),
 ('business_&_entrepreneurs', 6646),
 ('science_&_technology', 7499)]